In [ ]:
# Imports

%pip install seaborn
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df_wetter_dwd = pd.read_csv('additional-data/Klimadaten_DWD_Kiel_Holtenau.csv')



In [ ]:
# Diagnose & Vorbereitung: Vereinheitliche df_wetter_dwd und berechne Tagesmittel (Temperatur_DWD)
import os

# Falls df_wetter_dwd nicht definiert ist, versuche die Datei zu laden
if 'df_wetter_dwd' not in globals():
    filepath = 'additional-data/Klimadaten_DWD_Kiel_Holtenau.csv'
    if os.path.exists(filepath):
        print(f'Loading {filepath} into df_wetter_dwd')
        df_wetter_dwd = pd.read_csv(filepath)
    else:
        raise RuntimeError(f"df_wetter_dwd is not defined and file '{filepath}' not found")

# Zeige Spalten und Datentypen zur Diagnose
print('Columns in df_wetter_dwd:', list(df_wetter_dwd.columns))
print(df_wetter_dwd.dtypes)
display(df_wetter_dwd.head(5))

# Flexible Erkennung von Datums- und Temperaturspalte
date_candidates = ['MESS_DATUM', 'MESS_DATUM_UTC', 'Datum', 'date']
temp_candidates = ['TMK', 'Temperatur', 'Temperatur_DWD', 't']

date_col = next((c for c in date_candidates if c in df_wetter_dwd.columns), None)
temp_col = next((c for c in temp_candidates if c in df_wetter_dwd.columns), None)
if date_col is None:
    raise KeyError(f'No date column found. Available: {list(df_wetter_dwd.columns)}')
if temp_col is None:
    raise KeyError(f'No temperature column found. Available: {list(df_wetter_dwd.columns)}')

print(f'Using date_col={date_col}, temp_col={temp_col}')

# Wähle nur die relevanten Spalten und benenne für Konsistenz um
df_wetter_dwd = df_wetter_dwd[[date_col, temp_col]].copy()
df_wetter_dwd.rename(columns={date_col: 'Datum', temp_col: 'Temperatur_DWD'}, inplace=True)

# Datum robust konvertieren (MESS_DATUM oft YYYYMMDD als Zahl)
df_wetter_dwd['Datum'] = pd.to_datetime(df_wetter_dwd['Datum'].astype(str), format='%Y%m%d', errors='coerce')
mask_na_dates = df_wetter_dwd['Datum'].isna()
if mask_na_dates.any():
    # Fallback: generische Konvertierung für die verbleibenden
    df_wetter_dwd.loc[mask_na_dates, 'Datum'] = pd.to_datetime(df_wetter_dwd.loc[mask_na_dates, 'Datum'], errors='coerce')

# Normalisiere auf Tagesbeginn für sicheres Mapping
df_wetter_dwd['Datum'] = df_wetter_dwd['Datum'].dt.normalize()

print('NaT in Datum:', df_wetter_dwd['Datum'].isna().sum())
print('NaN in Temperatur_DWD:', df_wetter_dwd['Temperatur_DWD'].isna().sum())
display(df_wetter_dwd.head(10))

# Tagesmittel bilden (falls mehrere Messwerte pro Tag existieren)
temp_by_date = df_wetter_dwd.groupby('Datum')['Temperatur_DWD'].mean()
print('temp_by_date sample:')
print(temp_by_date.head(8))
print('df_wetter_dwd prepared.')